In [ ]:
# importation de quelques librairies

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
# chemin de nos données d'entrainement et de test

train_path = '../data/data_cla/stars_train_new.csv' # replace with your path
test_path = '../data/data_cla/stars_test_new.csv' # replace with your path

In [ ]:
# chargement des données

df_train = pd.read_csv(train_path).drop('obj_ID', axis=1)
df_test = pd.read_csv(test_path)

## Analyse des données

In [ ]:
# visualisation des données

df_train.head()

In [ ]:
# informations sur les données

df_train.info()

In [ ]:
# description des données

df_train.describe()

In [ ]:
# forme des données

print(df_train.shape, df_test.shape)

In [ ]:
# Disrtibution des classes

df_train['label'].hist()
plt.title('Star Class')
plt.xlabel('Class')
plt.xticks([0,1,2])

In [ ]:
# Matrice de corrélation

plt.figure(figsize=(10, 10))
sns.heatmap(df_train.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')

In [ ]:
# Distribution des variables

fig = plt.figure(figsize=(20, 20))
for i in range(len(df_train.columns)):
    fig.add_subplot(5, 3, i+1)
    sns.histplot(df_train.iloc[:, i], color='green', label=df_train.columns[i])
    # show the mean and median
    plt.axvline(df_train.iloc[:, i].mean(), linestyle='dashed', color='red', label='mean')
    plt.axvline(df_train.iloc[:, i].median(), linestyle='dashed', color='blue', label='median')
    plt.legend()

## Séparation train-test

In [ ]:
# Séparation des données en train test

X = df_train.drop('label', axis=1)
y = df_train['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Définition de notre espaces de paramètres

params = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Recherche des meilleurs paramètres
grid = GridSearchCV(rf, params, cv=5, n_jobs=-1, verbose=1)

grid.fit(X_train, y_train)

print(f"paramètres optimaux : {grid.best_params_}")

print(f"meilleur score : {grid.best_score_}")

# Prédiction sur les données de test

y_preds = grid.predict(X_test)

# Matrice de confusion

sns.heatmap(confusion_matrix(y_test, y_preds), annot=True, cmap='coolwarm')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/classif/random_forest.csv', index=False)

# soumission des résultats

rf = RandomForestClassifier(**grid.best_params_)

rf.fit(df_train.drop('label', axis=1), df_train['label'])

y_preds = rf.predict(df_test.drop('obj_ID', axis=1))

submission = pd.DataFrame({'obj_ID': df_test['obj_ID'],
                            'label': y_preds})

submission.to_csv('../soumission/classif/random_forest.csv', index=False)

## CatBoost Classifier

In [ ]:
from catboost import CatBoostClassifier

# Définition de notre modèle

cat = CatBoostClassifier(n_estimators=3000, max_depth=10, learning_rate=0.01)

# Entrainement du modèle

cat.fit(X_train, y_train, verbose=1)

# Prédiction sur les données de test

y_preds = cat.predict(X_test)

# Matrice de confusion

sns.heatmap(confusion_matrix(y_test, y_preds), annot=True, cmap='coolwarm')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds.ravel()})

new_results.to_csv('../résultats_models/classif/catboost.csv', index=False)

# soumission des résultats

cat = CatBoostClassifier(n_estimators=3000, max_depth=10, learning_rate=0.01)

cat.fit(df_train.drop('label', axis=1), df_train['label'], verbose=1)

y_preds = cat.predict(df_test.drop('obj_ID', axis=1))

submission = pd.DataFrame({'obj_ID': df_test['obj_ID'],
                            'label': y_preds.ravel()})

submission.to_csv('../soumission/classif/catboost.csv', index=False)


## Normalisation des données

In [ ]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

df_train_scaled = scaler.fit_transform(df_train.drop('label', axis=1))
df_test_scaled = scaler.transform(df_test.drop('obj_ID', axis=1))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# Définition de notre espaces de paramètres

params = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

# Recherche des meilleurs paramètres
grid = GridSearchCV(rf, params, cv=5, n_jobs=-1, verbose=1)

grid.fit(X_train_scaled, y_train)

print(f"paramètres optimaux : {grid.best_params_}")

print(f"meilleur score : {grid.best_score_}")

# Prédiction sur les données de test

y_preds = grid.predict(X_test_scaled)

# Matrice de confusion

sns.heatmap(confusion_matrix(y_test, y_preds), annot=True, cmap='coolwarm')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds})

new_results.to_csv('../résultats_models/classif/random_forest_scaled.csv', index=False)

# soumission des résultats

rf = RandomForestClassifier(**grid.best_params_)

rf.fit(df_train_scaled.drop('label', axis=1), df_train_scaled['label'])

y_preds = rf.predict(df_test_scaled.drop('obj_ID', axis=1))

submission = pd.DataFrame({'obj_ID': df_test_scaled['obj_ID'],
                            'label': y_preds})

submission.to_csv('../soumission/classif/random_forest_scaled.csv', index=False)

In [ ]:
from catboost import CatBoostClassifier

# Définition de notre modèle

cat = CatBoostClassifier(n_estimators=3000, max_depth=10, learning_rate=0.01)

# Entrainement du modèle

cat.fit(X_train_scaled, y_train, verbose=1)

# Prédiction sur les données de test

y_preds = cat.predict(X_test_scaled)

# Matrice de confusion

sns.heatmap(confusion_matrix(y_test, y_preds), annot=True, cmap='coolwarm')

# Sauvegarde des résultats pour streamlit

new_results = pd.DataFrame({'y_test': y_test,
                            'y_preds': y_preds.ravel()})

new_results.to_csv('../résultats_models/classif/catboost_scaled.csv', index=False)

# soumission des résultats

cat = CatBoostClassifier(n_estimators=3000, max_depth=10, learning_rate=0.01)

cat.fit(df_train_scaled.drop('label', axis=1), df_train_scaled['label'], verbose=1)

y_preds = cat.predict(df_test_scaled.drop('obj_ID', axis=1))

submission = pd.DataFrame({'obj_ID': df_test_scaled['obj_ID'],
                            'label': y_preds.ravel()})

submission.to_csv('../soumission/classif/catboost_scaled.csv', index=False)
